#Tải ollama và các model

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True)

In [ ]:
!ollama pull nomic-embed-text
!ollama pull qwen2


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 970aa74c0a90... 100% ▕▏ 274 MB                         
pulling c71d239df917... 100% ▕▏  11 KB                         
pulling ce4a164fc046... 100% ▕▏   17 B                         
pulling 31df23ea7daa... 100% ▕▏  420 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 43f7a214e532... 100% ▕▏ 4.4 GB                         
pulling 77c91b422cc9... 100% ▕▏ 1.4 KB                         
pulling c156170b718e... 100% ▕▏  11 KB                         
pulling f02dd72bb242... 100% ▕▏   59 B                         
pulling 75357d685f23... 100% ▕▏   28 B                         
pulling 648f809ced2b... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
!ollama list


NAME                       ID              SIZE      MODIFIED      
qwen2:latest               dd314f039b9d    4.4 GB    3 seconds ago    
nomic-embed-text:latest    0a109f422b47    274 MB    4 seconds ago    


In [ ]:
process2 = subprocess.Popen("ollama run qwen2", shell=True)

In [ ]:
print(f"Is `ollama serve` still running? {'Yes' if process.poll() is None else 'No'}")
print(f"Is `ollama run qwen2` still running? {'Yes' if process2.poll() is None else 'No'}")

Is `ollama serve` still running? Yes
Is `ollama run qwen2` still running? Yes


In [ ]:
!ollama ps

NAME            ID              SIZE      PROCESSOR    UNTIL              
qwen2:latest    dd314f039b9d    5.2 GB    100% CPU     4 minutes from now    


#Tải lightrag

In [ ]:
!pip install git+https://github.com/HKUDS/LightRAG.git

  Cloning https://github.com/HKUDS/LightRAG.git to /tmp/pip-req-build-jnqbel6z
  Running command git clone --filter=blob:none --quiet https://github.com/HKUDS/LightRAG.git /tmp/pip-req-build-jnqbel6z
  Resolved https://github.com/HKUDS/LightRAG.git to commit 598cc69d38db56030e17daecca66d7cee34a56ef
  Preparing metadata (setup.py) ... done
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


#Chạy file ví dụ để thu kết quả

In [ ]:
import asyncio
import nest_asyncio

nest_asyncio.apply()
import os
import inspect
import logging
from lightrag import LightRAG
from lightrag import QueryParam
from lightrag.llm.ollama import ollama_model_complete, ollama_embed
from lightrag.utils import EmbeddingFunc
from lightrag.kg.shared_storage import initialize_pipeline_status

WORKING_DIR = "./dickens"

logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)

if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)


async def initialize_rag():
    rag = LightRAG(
        working_dir=WORKING_DIR,
        llm_model_func=ollama_model_complete,
        llm_model_name="qwen2",
        llm_model_max_async=4,
        llm_model_max_token_size=2048,
        llm_model_kwargs={
            "host": "http://localhost:11434",
            "options": {"num_ctx": 2048},
        },
        embedding_func=EmbeddingFunc(
            embedding_dim=768,
            max_token_size=1024,
            func=lambda texts: ollama_embed(
                texts, embed_model="nomic-embed-text", host="http://localhost:11434"
            ),
        ),
    )

    await rag.initialize_storages()
    await initialize_pipeline_status()

    return rag


async def print_stream(stream):
    async for chunk in stream:
        print(chunk, end="", flush=True)


def main():
    # Initialize RAG instance
    rag = asyncio.run(initialize_rag())

    # Insert example text
    with open("book.txt", "r", encoding="utf-8") as f:
        rag.insert(f.read())

    # Test different query modes
    print("\nNaive Search:")
    print(
        rag.query(
            "Quyền dân sự được xác lập từ các căn cứ nào", param=QueryParam(mode="naive")
        )
    )

    print("\nLocal Search:")
    print(
        rag.query(
            "Quyền dân sự được xác lập từ các căn cứ nào?", param=QueryParam(mode="local")
        )
    )

    print("\nGlobal Search:")
    print(
        rag.query(
            "Quyền dân sự được xác lập từ các căn cứ nào?", param=QueryParam(mode="global")
        )
    )

    print("\nHybrid Search:")
    print(
        rag.query(
            "Quyền dân sự được xác lập từ các căn cứ nào?", param=QueryParam(mode="hybrid")
        )
    )

    # stream response
    resp = rag.query(
        "Quyền dân sự được xác lập từ các căn cứ nào?",
        param=QueryParam(mode="hybrid", stream=True),
    )

    if inspect.isasyncgen(resp):
        asyncio.run(print_stream(resp))
    else:
        print(resp)


if __name__ == "__main__":
    main()


INFO: Process 8302 Shared-Data created for Single Process
INFO: Process 8302 initialized updated flags for namespace: [full_docs]
INFO: Process 8302 ready to initialize storage namespace: [full_docs]
INFO: Process 8302 initialized updated flags for namespace: [text_chunks]
INFO: Process 8302 ready to initialize storage namespace: [text_chunks]
INFO: Process 8302 initialized updated flags for namespace: [entities]
INFO: Process 8302 initialized updated flags for namespace: [relationships]
INFO: Process 8302 initialized updated flags for namespace: [chunks]
INFO: Process 8302 initialized updated flags for namespace: [chunk_entity_relation]
INFO: Process 8302 initialized updated flags for namespace: [llm_response_cache]
INFO: Process 8302 ready to initialize storage namespace: [llm_response_cache]
INFO: Process 8302 initialized updated flags for namespace: [doc_status]
INFO: Process 8302 ready to initialize storage namespace: [doc_status]
INFO: Process 8302 storage namespace already initi


Naive Search:
Quyền dân sự được xác lập từ các căn cứ sau:

1. Hiến pháp và pháp luật: Quyền dân sự có thể dựa trên quy định của Hiến pháp và các luật khác của quốc gia hoặc khu vực cụ thể.

2. Hiến chương quốc tế: Các quyền dân sự cũng có thể được xác lập từ các điều khoản trong Hiến chương Liên Hợp Quốc, Công ước về Nhân quyền hay các hiệp định quốc tế liên quan đến nhân quyền.

3. Quyền tự nhiên của con người: Những quy tắc và quyền mà con người sinh ra đã sở hữu như quyền sống, quyền bình đẳng, quyền tự do cá nhân có thể được xem là cơ sở cho nhiều quyền khác trong luật dân sự.

4. Hiến pháp địa phương hoặc quốc gia: Nhiều nước sẽ có các hiến chương hoặc các điều khoản cụ thể của họ về quyền của người dân.

5. Quyền và nghĩa vụ hợp đồng: Trong hệ thống pháp lý ở một số quốc gia, quyền dân sự có thể được xác lập từ những quy tắc liên quan đến việc thực hiện các thỏa thuận hoặc hợp đồng.
   
6. Hiến chương địa phương hay quy định của tổ chức xã hội cộng đồng.

7. Quyền và nghĩa vụ t

In [ ]:
rag = asyncio.run(initialize_rag())
with open("new_book.txt", "r", encoding="utf-8") as f:
      rag.insert(f.read())
question = "Các chủ đề trong bài viết là gì?"
print("Naive Search: ", rag.query(question, param=QueryParam(mode="naive", stream=True)),
      "\n\nLocal Search: ", rag.query(question, param=QueryParam(mode="local", stream=False)),
      "\n\nGlobal Search: ", rag.query(question, param=QueryParam(mode="global", stream=True)),
      "\n\nHybrid Search: ", rag.query(question, param=QueryParam(mode="hybrid", stream=False)),
)

INFO: Process 8302 Shared-Data already initialized (multiprocess=False)
INFO: Process 8302 storage namespace already initialized: [full_docs]
INFO: Process 8302 storage namespace already initialized: [text_chunks]
INFO: Process 8302 storage namespace already initialized: [llm_response_cache]
INFO: Process 8302 storage namespace already initialized: [doc_status]
INFO: Process 8302 storage namespace already initialized: [full_docs]
INFO: Process 8302 storage namespace already initialized: [text_chunks]
INFO: Process 8302 storage namespace already initialized: [llm_response_cache]
INFO: Process 8302 storage namespace already initialized: [doc_status]


Naive Search:  Xin lỗi, tôi không thể cung cấp thông tin cụ thể về các chủ đề trong bài viết mà bạn đang hỏi vì tôi cần biết nội dung hoặc liên kết của bài viết đó. Tuy nhiên, nếu bạn cung cấp thêm thông tin, tôi sẽ rất vui lòng giúp bạn phân loại hoặc xác định các chủ đề chính của nó. 

Local Search:  Bài viết nói về một số khái niệm pháp lý liên quan đến trách nhiệm dân sự và vi phạm nghĩa vụ. Có hai khái niệm chính được đề cập đó là "lỗi cố ý" và "lỗi vô ý". 

"Lỗi cố ý" là trường hợp một người nhận thức rõ rằng hành động của mình có thể gây thiệt hại cho người khác nhưng vẫn thực hiện chúng, mong muốn hoặc không mong muốn sự thiệt hại xảy ra.

"Lỗi vô ý", ngược lại, là trường hợp một người không đoán trước được hậu quả và hậu quả có khả năng gây thiệt hại từ hành vi của họ, mặc dù phải biết hoặc có thể biết trước thiệt hại sẽ xảy ra hoặc thấy trước hành vi của mình có khả năng gây thiệt hại. 

Bài viết cũng đề cập đến việc khi một bên có lỗi trong nghĩa vụ mà gây thiệt hại cho bên 

In [ ]:
rag = asyncio.run(initialize_rag())
with open("new_book.txt", "r", encoding="utf-8") as f:
      rag.insert(f.read())
question = "Hãy tóm tắt các ý chính trong bài viết."
print("Naive Search: ", rag.query(question, param=QueryParam(mode="naive", stream=True)),
      "\n\nLocal Search: ", rag.query(question, param=QueryParam(mode="local", stream=False)),
      "\n\nGlobal Search: ", rag.query(question, param=QueryParam(mode="global", stream=True)),
      "\n\nHybrid Search: ", rag.query(question, param=QueryParam(mode="hybrid", stream=False)),
)

INFO: Process 8302 Shared-Data already initialized (multiprocess=False)
INFO: Process 8302 storage namespace already initialized: [full_docs]
INFO: Process 8302 storage namespace already initialized: [text_chunks]
INFO: Process 8302 storage namespace already initialized: [llm_response_cache]
INFO: Process 8302 storage namespace already initialized: [doc_status]
INFO: Process 8302 storage namespace already initialized: [full_docs]
INFO: Process 8302 storage namespace already initialized: [text_chunks]
INFO: Process 8302 storage namespace already initialized: [llm_response_cache]
INFO: Process 8302 storage namespace already initialized: [doc_status]


Naive Search:  Rất tiếc, nhưng tôi cần liên kết hoặc văn bản cụ thể để có thể tóm tắt các ý chính từ đó. Bạn có thể cung cấp thêm thông tin không? 

Local Search:  Bài viết đề cập đến vấn đề pháp lý liên quan đến trách nhiệm dân sự khi có vi phạm nghĩa vụ và gây ra thiệt hại, cũng như khái niệm về lỗi cố ý và lỗi vô ý theo luật dân sự.

Trong trường hợp vi phạm nghĩa vụ và có thiệt hại phát sinh do một phần lỗi của bên bị vi phạm, thì bên thực hiện vi phạm chỉ cần bồi thường thiệt hại tương ứng với mức độ lỗi của mình. 

Lỗi trong trách nhiệm dân sự được phân thành hai loại: lỗi cố ý và lỗi vô ý. Lỗi cố ý diễn ra khi người đó biết hoặc có thể biết trước rằng hành động của mình có khả năng gây ra thiệt hại cho người khác nhưng vẫn thực hiện hành vi và mong muốn thiệt hại xảy ra, hoặc chấp nhận thiệt hại dù không mong muốn.

Lỗi vô ý là trường hợp người đó không dự đoán được hành động của mình sẽ gây ra thiệt hại nhưng đã biết hoặc có thể biết trước rằng việc này có thể phát sinh, họ cho

In [ ]:
rag = asyncio.run(initialize_rag())
with open("new_book.txt", "r", encoding="utf-8") as f:
      rag.insert(f.read())
question = "Trong bài viết, tác giả giải thích thế nào về việc áp dụng tập quán?"
print("Naive Search: ", rag.query(question, param=QueryParam(mode="naive", stream=True)),
      "\n\nLocal Search: ", rag.query(question, param=QueryParam(mode="local", stream=False)),
      "\n\nGlobal Search: ", rag.query(question, param=QueryParam(mode="global", stream=True)),
      "\n\nHybrid Search: ", rag.query(question, param=QueryParam(mode="hybrid", stream=False)),
)

INFO: Process 8302 Shared-Data already initialized (multiprocess=False)
INFO: Process 8302 storage namespace already initialized: [full_docs]
INFO: Process 8302 storage namespace already initialized: [text_chunks]
INFO: Process 8302 storage namespace already initialized: [llm_response_cache]
INFO: Process 8302 storage namespace already initialized: [doc_status]
INFO: Process 8302 storage namespace already initialized: [full_docs]
INFO: Process 8302 storage namespace already initialized: [text_chunks]
INFO: Process 8302 storage namespace already initialized: [llm_response_cache]
INFO: Process 8302 storage namespace already initialized: [doc_status]


Naive Search:  Trong bài viết của mình, tác giả có thể đã giải thích việc áp dụng tập quán như sau:

Tập quán thường được hình thành và duy trì thông qua sự tái diễn liên tục trong thời gian dài. Việc áp dụng nó đòi hỏi một hành động lặp đi lặp lại một cách tự nhiên và không cần suy nghĩ nhiều, thường là do thói quen hoặc phản ứng điều kiện phản hồi.

Một số yếu tố có thể ảnh hưởng đến việc áp dụng tập quán bao gồm: sự thuận tiện của hành động, mức độ lợi ích mà tập quán mang lại, và cả tác động tâm lý xã hội (như cảm giác thỏa mãn khi thực hiện một tập quán nào đó).

Việc thay đổi hoặc giáo dục tập quán đòi hỏi kiên nhẫn và cố gắng liên tục để tạo ra những sự thay đổi mới trong hành vi. Điều này có thể đòi hỏi việc hiểu rõ nguyên nhân của tập quán cũ, xác định các trở ngại có thể xuất hiện khi áp dụng tập quán mới và cuối cùng là thực hiện một kế hoạch cụ thể để hướng dẫn quá trình thay đổi.

Tóm lại, tác giả có thể đã đề cập rằng việc áp dụng tập quán đòi hỏi một sự hiểu biết sâu sắc

#In đồ thị

In [ ]:
!pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.2 MB/s eta 0:00:00


In [ ]:
import pipmaster as pm

if not pm.is_installed("pyvis"):
    pm.install("pyvis")
if not pm.is_installed("networkx"):
    pm.install("networkx")

import networkx as nx
from pyvis.network import Network
import random

# Load the GraphML file
G = nx.read_graphml("./dickens/graph_chunk_entity_relation.graphml")

# Create a Pyvis network
net = Network(height="100vh", notebook=True)

# Convert NetworkX graph to Pyvis network
net.from_nx(G)


# Add colors and title to nodes
for node in net.nodes:
    node["color"] = "#{:06x}".format(random.randint(0, 0xFFFFFF))
    if "description" in node:
        node["title"] = node["description"]

# Add title to edges
for edge in net.edges:
    if "description" in edge:
        edge["title"] = edge["description"]

# Save and display the network
net.show("knowledge_graph.html")


knowledge_graph.html
